# SpeakSense - Language Detection System (Machine Learning CSCI 6364)

**Abde Manaaf Ghadiali (G29583342), Gehna Ahuja (G00000000), Venkatesh Shanmugam (G00000000)**

The objective of this project is to develop a robust and accurate system capable of detecting the language spoken in audio recordings. By leveraging advanced machine learning algorithms and signal processing techniques, the system aims to accurately identify the language spoken in various audio inputs, spanning diverse accents, dialects, and environmental conditions. This language detection solution seeks to provide practical applications in speech recognition, transcription, translation, and other fields requiring language-specific processing, thereby enhancing accessibility and usability across linguistic boundaries.

This code sets up an environment for working with audio data, particularly focusing on Indian languages. Here's a breakdown of what each part does:

1. **Importing Libraries**: Imports necessary libraries for data manipulation, visualization, machine learning, and audio processing.

2. **Setting Display Options and Suppressing Warnings**: Configures display options for Pandas and suppresses warnings.

3. **Setting Random Seed**: Sets a random seed for reproducibility.

4. **Downloading Datasets**: Checks if the necessary datasets are downloaded, and if not, downloads them from Kaggle using the OpenDatasets library and organizes them into appropriate directories.

5. **Audio Data Processing**: Prepares the audio data for further analysis. This might include feature extraction, preprocessing, and organizing the data for training machine learning models.

6. **Machine Learning**: Utilizes machine learning techniques for tasks such as spoken language identification. This involves splitting the data into training and testing sets, building machine learning models (such as Random Forest or Gradient Boosting), evaluating the models, and generating classification reports and confusion matrices.

7. **Deep Learning**: Utilizes deep learning techniques, specifically convolutional neural networks (CNNs), for tasks such as spoken language identification. This involves building and training deep learning models using the TensorFlow and Keras libraries.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import tensorflow as tf
import soundfile as sf

import librosa
import os
import glob
import warnings

from tqdm import tqdm

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
np.random.seed(42)

# Feature Extraction

### Audio Chunking

In [2]:
language_dataframe = pd.read_csv('../data/model_data/language_dataframe_v1.csv')
language_dataframe = language_dataframe[language_dataframe['audio_duration_sec'] >= 5]

spoken_languages_train_path_dataset = '../data/spoken_language_identification/train/train/'
indian_languages_train_path_dataset = '../data/model_data/chunked_audio/*.wav'

language_dataframe['language_label'].unique()

array(['german', 'english', 'spanish', 'bengali', 'gujarati', 'hindi',
       'kannada', 'malayalam', 'marathi', 'tamil', 'telugu', 'urdu'],
      dtype=object)

This code processes audio data for Indian languages by chunking the audio files into segments of fixed duration and resampling them to a target sample rate. Here's a breakdown:

1. **Defining Parameters**:
    - `indian_languages`: A list containing the names of Indian languages.
    - `target_sample_rate`: The target sample rate to which the audio will be resampled.
    - `audio_duration`: The duration (in seconds) of each audio chunk.

2. **Processing Audio Files**:
    - It iterates over each Indian language in the `indian_languages` list.
    - For each language, it creates a subset DataFrame `lang_dataframe` containing only files for that language.
    - It initializes an empty list `audio_data_list` to store audio data for chunking and a variable `chunk_count` to track the number of chunks processed.

3. **Chunking Audio Files**:
    - It iterates over each file path in `lang_dataframe['file_name']`.
    - It loads each audio file using `librosa.load()` with the specified target sample rate.
    - It appends the loaded audio data to the `audio_data_list`.
    - If the length of `audio_data_list` reaches 1500 (a chosen threshold) or if it's the last file in the subset DataFrame, it proceeds to chunk the audio data.
    - It uses TensorFlow's `tf.signal.frame()` function to segment the concatenated audio data into fixed-duration chunks.
    - It iterates over each chunk and writes it to a FLAC file in a specified directory, with the filename indicating the chunk number, index within the language, and language name.
    - It increments `chunk_count` to track the number of processed chunks and resets `audio_data_list` for the next iteration or language.

Overall, this code efficiently processes audio files for Indian languages, chunks them into fixed-duration segments, and saves them to disk for further analysis or use in machine learning models.


In [3]:
language_dataframe

,file_name,language_label,file_size_kb,sample_rate,audio_duration_sec
0,../data/spoken_language_identification/train/t...,german,241.913,22050.0,10.0
1,../data/spoken_language_identification/train/t...,german,247.070,22050.0,10.0
2,../data/spoken_language_identification/train/t...,german,235.787,22050.0,10.0
3,../data/spoken_language_identification/train/t...,german,236.962,22050.0,10.0
4,../data/spoken_language_identification/train/t...,german,239.393,22050.0,10.0
...,...,...,...,...,...
303663,../data/audio_dataset_indian_languages/Languag...,urdu,78.915,44100.0,5.0
303664,../data/audio_dataset_indian_languages/Languag...,urdu,78.915,44100.0,5.0
303667,../data/audio_dataset_indian_languages/Languag...,urdu,78.915,44100.0,5.0
303669,../data/audio_dataset_indian_languages/Languag...,urdu,78.794,48000.0,5.0


In [4]:
indian_languages = ['bengali', 'gujarati', 'hindi', 'kannada', 'malayalam', 'marathi', 'tamil', 'telugu', 'urdu']
target_sample_rate = 22050
audio_duration = 10

for lang in indian_languages:
    lang_dataframe = language_dataframe[language_dataframe['language_label'] == lang].copy()
    audio_data_list = []
    chunk_count = 0

    for idx, file_path in tqdm(enumerate(lang_dataframe['file_name'].values), desc=f'Processing {lang}', total=len(lang_dataframe)):
        audio_data_list.append(librosa.load(file_path, sr=target_sample_rate)[0])

        if (len(audio_data_list) >= 1500) or (idx == len(lang_dataframe) - 1):
            audio_chunks = tf.signal.frame(np.concatenate(audio_data_list), int(audio_duration * target_sample_rate), int(audio_duration * target_sample_rate), pad_end=True)

            for chunk_idx, audio_chunk in enumerate(audio_chunks):
                if chunk_idx == len(audio_chunks) - 1:
                    continue

                sf.write(file=f'../data/model_data/chunked_audio/{chunk_count}_{chunk_idx}_{lang}.wav', data=np.ravel(audio_chunk), samplerate=target_sample_rate, format='wav', subtype='FLOAT')

            chunk_count += 1
            audio_data_list = []

Processing urdu: 100%|██████████| 11727/11727 [01:28<00:00, 132.40it/s]


In [5]:
data_bengali, sample_rate_bengali = librosa.load('../data/model_data/chunked_audio/0_0_bengali.wav')
print(f'Audio Data Sample Rate: {sample_rate_bengali}')
print(f'Audio Data: {data_bengali}')

ipd.Audio(data=data_bengali, rate=sample_rate_bengali)

Audio Data Sample Rate: 22050
Audio Data: [-1.8423570e-14 -6.3668206e-15 -4.9556743e-15 ... -2.5771603e-02
 -2.7053937e-02 -2.6320292e-02]


### Re-Calculate Descriptive Statistics

In [6]:
def load_data(file_name: str) -> tuple:
    """Loads audio data from a file using librosa.

    Parameters:
        file_name (str): Path to the audio file.

    Returns:
        tuple: A tuple containing the sample rate and duration of the audio in seconds.
               If an error occurs during processing, returns (np.nan, np.nan).
    """

    try:
        # Load audio data and sample rate
        audio_data, sample_rate = librosa.load(file_name, sr=None)

        # Calculate duration of audio in seconds
        audio_duration_sec = int(librosa.get_duration(y=audio_data, sr=sample_rate))

        return (sample_rate, audio_duration_sec)

    except Exception as e:
        # Print error message if an exception occurs during processing
        print(f"Error processing {file_name}: {str(e)}")

        # Return NaN values for sample rate and duration
        return (np.nan, np.nan)

In [7]:
spoken_language_dataframe = pd.DataFrame({'file_name': [spoken_languages_train_path_dataset + file_name for file_name in os.listdir(spoken_languages_train_path_dataset)]})
spoken_language_dataframe['language_label'] = spoken_language_dataframe['file_name'].str.split('/', expand=True).iloc[:, 5].str.split('_', expand=True).iloc[:, 0].replace(to_replace={
    'de': 'german', 'en': 'english', 'es': 'spanish'})

indian_language_dataframe = pd.DataFrame({'file_name': glob.glob(indian_languages_train_path_dataset)})
indian_language_dataframe['language_label'] = indian_language_dataframe['file_name'].str.split('_', expand=True).iloc[:, 4].str.split('.', expand=True).iloc[:, 0].str.lower()

language_dataframe = pd.concat([spoken_language_dataframe, indian_language_dataframe], ignore_index=True)
language_dataframe['file_size_kb'] = (language_dataframe['file_name'].apply(lambda x: os.path.getsize(x)) / 1024).round(3)

language_dataframe.to_csv('../data/model_data/language_dataframe_v2.csv', index=False)

for lang in tqdm(language_dataframe['language_label'].unique(), desc="Languages"):
    lang_data = language_dataframe[language_dataframe['language_label'] == lang].copy()
    lang_data[['sample_rate', 'audio_duration_sec']] = lang_data['file_name'].apply(lambda file_name: pd.Series(load_data(file_name=file_name)))

    lang_data.to_csv(f'../data/model_data/data_subset/language_dataframe_{lang}_v2.csv', index=False)

language_dataframe = pd.concat([pd.read_csv(f'../data/model_data/data_subset/language_dataframe_{lang}_v2.csv') for lang in language_dataframe['language_label'].unique()], ignore_index=True).dropna()
language_dataframe.to_csv('../data/model_data/language_dataframe_v2.csv', index=False)

language_dataframe

Languages: 100%|██████████| 12/12 [03:04<00:00, 15.35s/it]


,file_name,language_label,file_size_kb,sample_rate,audio_duration_sec
0,../data/spoken_language_identification/train/t...,german,241.913,22050,10
1,../data/spoken_language_identification/train/t...,german,247.070,22050,10
2,../data/spoken_language_identification/train/t...,german,235.787,22050,10
3,../data/spoken_language_identification/train/t...,german,236.962,22050,10
4,../data/spoken_language_identification/train/t...,german,239.393,22050,10
...,...,...,...,...,...
115719,../data/model_data/chunked_audio\7_96_urdu.wav,urdu,861.406,22050,10
115720,../data/model_data/chunked_audio\7_97_urdu.wav,urdu,861.406,22050,10
115721,../data/model_data/chunked_audio\7_98_urdu.wav,urdu,861.406,22050,10
115722,../data/model_data/chunked_audio\7_99_urdu.wav,urdu,861.406,22050,10


In [8]:
language_dataframe['language_label'].value_counts()

language_label
german       24360
english      24360
spanish      24360
urdu          5864
gujarati      5084
bengali       4938
hindi         4734
tamil         4728
marathi       4665
telugu        4324
malayalam     4276
kannada       4031
Name: count, dtype: int64

In [9]:
language_dataframe['audio_duration_sec'].unique()

array([10], dtype=int64)

In [10]:
language_dataframe_v1 = pd.read_csv('../data/model_data/language_dataframe_v1.csv')
language_dataframe_v2 = pd.read_csv('../data/model_data/language_dataframe_v2.csv')

In [11]:
language_dataframe_v1.describe().round(2)

,file_size_kb,sample_rate,audio_duration_sec
count,303674.00,303674.00,303674.00
mean,108.21,39540.44,5.72
std,60.91,10887.05,2.45
min,0.53,22050.00,0.00
25%,78.42,22050.00,4.00
50%,78.51,44100.00,5.00
75%,78.92,48000.00,5.00
max,857.51,48000.00,10.00


In [12]:
language_dataframe_v2.describe().round(2)

,file_size_kb,sample_rate,audio_duration_sec
count,115724.00,115724.0,115724.0
mean,451.49,22050.0,10.0
std,313.76,0.0,0.0
min,101.11,22050.0,10.0
25%,205.39,22050.0,10.0
50%,236.07,22050.0,10.0
75%,861.41,22050.0,10.0
max,861.41,22050.0,10.0


In [13]:
language_dataframe_v1['audio_duration_sec'].value_counts()

audio_duration_sec
4.0     144833
5.0      85293
10.0     73080
0.0        154
1.0        108
2.0        107
3.0         99
Name: count, dtype: int64

In [14]:
language_dataframe_v2['audio_duration_sec'].value_counts()

audio_duration_sec
10    115724
Name: count, dtype: int64

In [15]:
lang_value_counts = pd.merge(
    left=pd.DataFrame(language_dataframe_v1['language_label'].value_counts()).reset_index(),
    right=pd.DataFrame(language_dataframe_v2['language_label'].value_counts()).reset_index(),
    on='language_label').rename(columns={'language_label': 'Language Labels', 'count_x': 'Before Chunking', 'count_y': 'After Chunking'})

lang_value_counts

,Language Labels,Before Chunking,After Chunking
0,urdu,31958,5864
1,bengali,27258,4938
2,gujarati,26436,5084
3,hindi,25462,4734
4,marathi,25378,4665
5,german,24360,24360
6,english,24360,24360
7,spanish,24360,24360
8,tamil,24195,4728
9,malayalam,24044,4276


### MFCC Mean Data Creation

In [16]:
language_dataframe = pd.read_csv('../data/model_data/language_dataframe_v2.csv')

This code calculates the mean Mel-frequency cepstral coefficients (MFCC) features for audio files and saves them in CSV format. Here's the breakdown:

1. **Defining Function to Calculate MFCC Features**:
    - The function `get_mfcc_features_mean` takes a file name as input and returns the mean MFCC features.
    - It loads the audio data using `librosa.load()` with a sample rate of 22050 Hz.
    - It calculates MFCC features using `librosa.feature.mfcc()` with parameters specifying 40 coefficients (`n_mfcc=40`).
    - It calculates the mean of MFCC features over time and returns the result.

2. **Processing Audio Files and Calculating MFCC Features**:
    - It iterates over each unique language label in the `language_dataframe`.
    - For each language, it creates a subset DataFrame `lang_data` containing only files for that language.
    - It calculates the MFCC features mean for each audio file in the subset using the `get_mfcc_features_mean` function and applies it to the `'file_name'` column using `apply()`.
    - It saves the resulting DataFrame to a CSV file named `mfcc_data_{lang}_v1.csv` in a specified directory.

3. **Combining Processed DataFrames and Saving to CSV**:
    - It reads each CSV file for each language subset and concatenates them into a single DataFrame `mfcc_feature_mean_dataframe`.
    - It drops any rows with NaN values.
    - It saves the combined DataFrame to a CSV file named `mfcc_feature_mean_dataframe_v1.csv`.


In [18]:
def extract_features_mean(file_name: str) -> np.ndarray:
    audio_data, sample_rate = librosa.load(file_name)

    mfcc_feat = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
    zero_crossing_rate_feat = librosa.feature.zero_crossing_rate(audio_data)

    mag_s, _ = librosa.magphase(librosa.stft(audio_data))
    spectral_rolloff_feat = librosa.feature.spectral_rolloff(S=mag_s, sr=sample_rate)

    onset_strength_feat = librosa.onset.onset_strength(y=audio_data, sr=sample_rate).reshape(1, 431)
    rms_feat = librosa.feature.rms(y=audio_data)

    power_spec = np.abs(librosa.stft(audio_data))**2
    norm_power_spec = power_spec / np.sum(power_spec, axis=0, keepdims=True)
    spectral_entropy_feat = -np.sum(norm_power_spec * np.log2(norm_power_spec), axis=0).reshape(1, 431)

    chroma_stft_feat = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)

    pitch_feat, _ = librosa.piptrack(y=audio_data, sr=sample_rate)
    pitch_feat = pitch_feat.mean(axis=0).reshape(1, 431)

    features_combined = np.concatenate([
        mfcc_feat, zero_crossing_rate_feat, spectral_rolloff_feat, onset_strength_feat, rms_feat,
        spectral_entropy_feat, chroma_stft_feat, pitch_feat], axis=0)

    features_combined[np.isnan(features_combined)] = 0

    return list(np.round(np.mean(features_combined.T, axis=0), 5))


language_dataframe = language_dataframe.groupby('language_label').apply(lambda row: row.sample(n=4000)).reset_index(drop=True)

for lang in language_dataframe['language_label'].unique():
    print(f'Currently Running: {lang}')

    lang_data = language_dataframe[language_dataframe['language_label'] == lang].copy()
    feature_dataframe = lang_data['file_name'].apply(lambda file_name: pd.Series(extract_features_mean(file_name=file_name)))
    feature_dataframe['language_label'] = lang

    feature_dataframe.to_parquet(f'../data/model_data/mfcc_mean_dataframes/mfcc_data_{lang}_v1.parquet', index=False)

mfcc_feature_mean_dataframe = pd.concat([pd.read_parquet(f'../data/model_data/mfcc_mean_dataframes/mfcc_data_{lang}_v1.parquet') for lang in language_dataframe['language_label'].unique()], ignore_index=True).dropna()
mfcc_feature_mean_dataframe.to_parquet('../data/model_data/mfcc_feature_mean_dataframe_v1.parquet', index=False)

Currently Running: german
Currently Running: english
Currently Running: spanish
Currently Running: bengali
Currently Running: gujarati
Currently Running: hindi
Currently Running: kannada
Currently Running: malayalam
Currently Running: marathi
Currently Running: tamil
Currently Running: telugu
Currently Running: urdu


### MFCC Matrix Data Creation

In [19]:
language_dataframe = pd.read_csv('../data/model_data/language_dataframe_v2.csv')

This code calculates the Mel-frequency cepstral coefficients (MFCC) features for audio files and saves them in CSV format. Here's the breakdown:

1. **Defining Function to Calculate MFCC Features**:
    - The function `get_mfcc_features` takes a file name as input and returns the MFCC features as a list.
    - It loads the audio data using `librosa.load()` with a sample rate of 22050 Hz.
    - It calculates MFCC features using `librosa.feature.mfcc()` with parameters specifying 40 coefficients (`n_mfcc=40`).
    - It converts the MFCC features to a list and returns them.

2. **Sampling Data**:
    - It downsamples the `language_dataframe` by randomly selecting 500 samples for each language.
    - This downsampling is done to reduce computational complexity and ensure balanced representation across languages.

3. **Processing Audio Files and Calculating MFCC Features**:
    - It iterates over each unique language label in the downsampled `language_dataframe`.
    - For each language, it creates a subset DataFrame `lang_data` containing a random sample of 500 files for that language.
    - It calculates the MFCC features for each audio file in the subset using the `get_mfcc_features` function and applies it to the `'file_name'` column using `apply()`.
    - It saves the resulting DataFrame to a CSV file named `mfcc_data_{lang}_v1.csv` in a specified directory.

4. **Combining Processed DataFrames and Saving to CSV**:
    - It reads each CSV file for each language subset and concatenates them into a single DataFrame `mfcc_feature_dataframe`.
    - It drops any rows with NaN values.
    - It saves the combined DataFrame to a CSV file named `mfcc_feature_dataframe_v1.csv`.


In [20]:
def extract_features(file_name: str) -> np.ndarray:
    audio_data, sample_rate = librosa.load(file_name)

    mfcc_feat = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
    zero_crossing_rate_feat = librosa.feature.zero_crossing_rate(audio_data)

    mag_s, _ = librosa.magphase(librosa.stft(audio_data))
    spectral_rolloff_feat = librosa.feature.spectral_rolloff(S=mag_s, sr=sample_rate)

    onset_strength_feat = librosa.onset.onset_strength(y=audio_data, sr=sample_rate).reshape(1, 431)
    rms_feat = librosa.feature.rms(y=audio_data)

    power_spec = np.abs(librosa.stft(audio_data))**2
    norm_power_spec = power_spec / np.sum(power_spec, axis=0, keepdims=True)
    spectral_entropy_feat = -np.sum(norm_power_spec * np.log2(norm_power_spec), axis=0).reshape(1, 431)

    chroma_stft_feat = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)

    pitch_feat, _ = librosa.piptrack(y=audio_data, sr=sample_rate)
    pitch_feat = pitch_feat.mean(axis=0).reshape(1, 431)

    features_combined = np.round(np.concatenate([
        mfcc_feat, zero_crossing_rate_feat, spectral_rolloff_feat, onset_strength_feat, rms_feat,
        spectral_entropy_feat, chroma_stft_feat, pitch_feat], axis=0), 5)

    features_combined[np.isnan(features_combined)] = 0

    return np.round(features_combined, 5).flatten()


language_dataframe = language_dataframe.groupby('language_label').apply(lambda row: row.sample(n=1000)).reset_index(drop=True)

for lang in language_dataframe['language_label'].unique():
    print(f'Currently Running: {lang}')

    lang_data = language_dataframe[language_dataframe['language_label'] == lang].copy()
    feature_dataframe = lang_data['file_name'].apply(lambda file_name: pd.Series(extract_features(file_name=file_name)))
    feature_dataframe['language_label'] = lang

    feature_dataframe.to_parquet(f'../data/model_data/mfcc_dataframes/mfcc_data_{lang}_v1.parquet', index=False)

mfcc_feature_dataframe = pd.concat([pd.read_parquet(f'../data/model_data/mfcc_dataframes/mfcc_data_{lang}_v1.parquet') for lang in language_dataframe['language_label'].unique()], ignore_index=True).dropna()
mfcc_feature_dataframe.to_parquet('../data/model_data/mfcc_feature_dataframe_v1.parquet', index=False)

Currently Running: bengali
Currently Running: english
Currently Running: german
Currently Running: gujarati
Currently Running: hindi
Currently Running: kannada
Currently Running: malayalam
Currently Running: marathi
Currently Running: spanish
Currently Running: tamil
Currently Running: telugu
Currently Running: urdu


### Spectogram Images

In [19]:
language_dataframe = pd.read_csv('../data/model_data/language_dataframe_v2.csv')

This code snippet defines a function `save_audio_spectrogram_plot` and uses it to generate and save spectrogram images for audio files. Here's a breakdown:

1. **Defining Function to Save Spectrogram**:
    - The function `save_audio_spectrogram_plot` takes four parameters: `file_name` (path to the audio file), `idx` (index of the file), `label` (language label of the file), and `figsize` (optional tuple specifying the size of the figure, default is (5, 5)).
    - It loads the audio data using `librosa.load()` with the original sample rate.
    - It creates a new figure with a specified size and no frame.
    - It generates the spectrogram plot using `plt.specgram()`, which computes and plots the spectrogram of the audio data.
    - It saves the generated spectrogram plot as a PNG image in a directory corresponding to the language label and with a filename based on the index.

2. **Processing Audio Files and Generating Spectrograms**:
    - It first downsamples the `language_dataframe` by randomly selecting 500 samples for each language. This is done to reduce computational load and ensure balanced representation across languages.
    - It iterates over each unique language label in the downsampled `language_dataframe`.
    - For each language, it creates a subset DataFrame `lang_data` containing a random sample of 500 files for that language.
    - It iterates over each file in `lang_data['file_name']`.
    - If the directory corresponding to the language label does not exist, it creates one.
    - It calls the `save_audio_spectrogram_plot` function to generate and save the spectrogram image for each audio file in the language subset.


In [21]:
def save_audio_spectrogram_plot(file_name: str, idx: int, label: str, figsize: tuple = (8, 8)):
    audio_data, sample_rate = librosa.load(file_name, sr=None)

    fig = plt.figure(figsize=figsize, frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    plt.specgram(audio_data, Fs=sample_rate)

    fig.savefig(f'../data/model_data/spectrogram_images/{label}/{idx}.png', dpi=128)
    plt.close()

language_dataframe = language_dataframe.groupby('language_label').apply(lambda row: row.sample(n=200)).reset_index(drop=True)

for lang in language_dataframe['language_label'].unique():
    lang_data = language_dataframe[language_dataframe['language_label'] == lang].copy()

    for idx, file_path in tqdm(enumerate(lang_data['file_name'].values), total=len(lang_data), desc=f"Processing {lang}"):
        if not os.path.exists(f'../data/model_data/spectrogram_images/{lang}'):
            os.makedirs(f'../data/model_data/spectrogram_images/{lang}')

        save_audio_spectrogram_plot(file_path, idx, lang)

Processing bengali:   0%|          | 0/200 [00:00<?, ?it/s]

Processing urdu: 100%|██████████| 200/200 [00:52<00:00,  3.82it/s]
